In [2]:
import pandas as pd
import regex as re
import csv
import os

In [3]:
df = pd.read_csv('Total_Production.csv')
pd.set_option('display.max_columns', None)
df


,Total production by all exporting countries,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31
0,In thousand 60kg bags,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Crop year,NaN,1990/91,1991/92,1992/93,1993/94,1994/95,1995/96,1996/97,1997/98,1998/99,1999/00,2000/01,2001/02,2002/03,2003/04,2004/05,2005/06,2006/07,2007/08,2008/09,2009/10,2010/11,2011/12,2012/13,2013/14,2014/15,2015/16,2016/17,2017/18,2018/19,2019/20
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,April group,NaN,"40,795","42,530","46,868","40,167","39,990","29,396","44,298","39,476","50,892","60,964","44,389","43,821","61,378","41,168","53,794","48,116","63,836","52,917","68,552","61,568","71,811","64,395","75,896","74,494","69,873","71,332","75,508","70,324","81,708","76,076"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62,Viet Nam,(R/A),"1,310","1,438","2,340","3,020","3,632","3,970","5,752","6,979","7,052","11,631","14,841","13,093","11,574","15,337","14,370","13,842","19,340","16,405","18,438","17,825","20,000","26,500","23,402","27,610","27,241","31,090","27,819","33,432","30,283","30,487"
63,Yemen,(A),0,0,0,0,0,0,0,0,69,61,64,62,57,193,206,203,207,198,220,135,161,185,188,187,178,156,143,132,86,91
64,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
65,Total,NaN,"93,230","101,267","98,524","91,761","93,314","87,321","103,308","99,887","108,866","131,383","113,746","107,902","122,625","105,503","116,078","111,169","135,400","121,842","134,800","127,873","140,078","141,327","151,184","153,910","150,302","156,126","162,320","163,693","172,461","165,053"


In [4]:
pd.DataFrame(df.loc[2]).head()


,2
Total production by all exporting countries,Crop year
Unnamed: 1,NaN
Unnamed: 2,1990/91
Unnamed: 3,1991/92
Unnamed: 4,1992/93


In [5]:
# The comment lines below will change the column year format from 1990/91 to 1990. The difference and something that is important to know is that the crop year begins at the month of 
# every group month. Let's said, Colombia's crop year begins on October 1990, then the crop year will go through october 1991, which slice distinguish from the current calendar year.
# So, depend on what you need from this column, it can be change the format from the column.

def CleaningCropYear(df,columname,rows):

    april_index = df.loc[5:18].index.values.astype(int)
    july_index = df.loc[21:27].index.values.astype(int)
    october_index = df.loc[30:63].index.values.astype(int)
    df['Crop Group'] = 'Crop Group'
    for i in range(0,len(df)):
        for ai in april_index:
            if i == ai:
                df.loc[:,'Crop Group'][i] = 'April Group'
                break
            else:
                continue
        for ji in july_index:
            if i == ji:
                df.loc[:,'Crop Group'][i] = 'July Group'
                break
            else:
                continue
        for oi in october_index:
            if i == oi:
                df.loc[:,'Crop Group'][i] = 'October Group'
                break
            else:
                continue
    df = df.fillna('99')
    years = pd.DataFrame(df.loc[2])
    years_names = years[2]
    
    columns_toModify = df.columns
    
    # for i in range(0,len(df.loc[2])):
    #     years_names[i] = re.sub(r'\/[\d]{2}$','',df.loc[2][i])
    years_names[1] = 'Type of Coffee'
    rango = len(columns_toModify)
    for i in range(0,rango):
        df = df.rename(columns={columns_toModify[i]:years_names[i]})

    df = df.drop(rows)
    df = df.reset_index().drop('index',axis=1)
    df = df.rename(columns={df.columns[0]:'Countries'})
    df = df.melt(id_vars=[df.columns[0],df.columns[1],df.columns[32]],var_name='Crop Year',value_name=columname)
    df['Countries'] = df['Countries'].str.strip()     
    index = 0

    for production in df[columname]:
        df[columname][index] = re.sub(r',','',production)
        index+=1
        
    df[columname] = df[columname].astype('int64')
    df['Type of Coffee'] = df['Type of Coffee'].replace('(R)','Robusta')
    df['Type of Coffee'] = df['Type of Coffee'].replace('(A)','Arabica')
    df['Type of Coffee'] = df['Type of Coffee'].replace('(R/A)','Robustica/Arabica')
    df['Type of Coffee'] = df['Type of Coffee'].replace('(A/R)','Arabica/Robusta')

    return df

def CleaningCalendarYear(df,column_name, rows):
    years = pd.DataFrame(df.loc[2])
    years_names = years[2]
    index = len(df.columns)
    columns_toModify = df.columns
    
    for i in range(0,index):
        df = df.rename(columns={columns_toModify[i]:years_names[i]})
    df = df.drop(rows)
    df = df.reset_index().drop('index',axis=1)
    df = df.rename(columns={df.columns[0]:'Countries'})
    df = df.melt(id_vars=['Countries'],var_name='Year',value_name=column_name)
    df['Countries'] = df['Countries'].str.strip()
    df = df.fillna('999')

    index = 0
    for production in df[column_name]:
        df[column_name][index] = re.sub(r',','',production)
        index+=1
    
    df[column_name] = df[column_name].astype('int64')

    return df

def CleaningCalendarYearGrowers(df,column_name, rows):
    years = pd.DataFrame(df.loc[2])
    years_names = years[2].astype('string')
    index = len(df.columns)
    columns_toModify = df.columns
    j = 0
    for j in range(0,len(years_names)):
        years_names[j] = re.sub(r'.0$','',years_names[j])
        j+=1
    
    for i in range(0,index-1):
         df = df.rename(columns={columns_toModify[i]:years_names[i]})
    df = df.drop(rows)
    df = df.reset_index().drop('index',axis=1)
    df = df.rename(columns={df.columns[0]:'Countries'})
    df = df.melt(id_vars=['Countries','Type of Coffee'],var_name='Year',value_name=column_name)
    df['Countries'] = df['Countries'].str.strip()
    df = df.fillna('0')

    return df

def CleaningCalendarYearRetail(df,column_name, rows):
    years = pd.DataFrame(df.loc[2])
    years_names = years[2].astype('string')
    index = len(df.columns)
    columns_toModify = df.columns
    j = 0
    for j in range(0,len(years_names)):
        years_names[j] = re.sub(r'.0$','',years_names[j])
        j+=1
    
    for i in range(0,index):
         df = df.rename(columns={columns_toModify[i]:years_names[i]})
    df = df.drop(rows)
    df = df.reset_index().drop('index',axis=1)
    df = df.rename(columns={df.columns[0]:'Countries'})
    df = df.melt(id_vars=['Countries'],var_name='Year',value_name=column_name)
    df['Countries'] = df['Countries'].str.strip()
    df = df.fillna('0')

    return df

In [6]:
df_totalProduction = CleaningCropYear(df,'Total Production (x1000 60kg) bags',[0,1,2,3,4,19,20,28,29,64,65,66])


In [7]:
df1 = pd.read_csv('Domestic_Consumption.csv')
df_DomesticConsumption = CleaningCropYear(df1,'Domestic Consumption (x1000 60kg) bags',[0,1,2,3,4,19,20,28,29,64,65,66])


In [8]:
df2 = pd.read_csv('Gross_Opening_stock.csv')
df_grossOpeningStock = CleaningCropYear(df2,'Gross Opening Stock (x1000 60kg) bags',[0,1,2,3,4,19,20,28,29,64,65,66])

In [9]:
df3 = pd.read_csv('ExportsCropyear.csv')
df_ExportsCropYear = CleaningCropYear(df3,'Exports Crop Year in (x1000 60kg) bags',[0,1,2,3,4,19,20,28,29,64,65,66])


In [10]:
df4 = pd.read_csv('Exports.csv')
df_ExportsCalendarYear = CleaningCalendarYear(df4,'Exports of all forms of Coffee in (x1000 60kg) bags',[0,1,2,58,59,60])

In [11]:
df5 = pd.read_csv('Import.csv')
df_Imports = CleaningCalendarYear(df5,'Imports in (x1000 60kg) bags',[0,1,2,3,4,33,41,42,43])

In [12]:
df6 = pd.read_csv('ReExports.csv')
df_ReExports = CleaningCalendarYear(df6,'Re-exports by importing countries in (x1000 60kg) bags',[0,1,2,3,4,33,41,42,43])

In [ ]:
df7 = pd.read_csv('Price_toGrowers.csv')

iterator = df7.index.values.astype(int)
index_toChange = df7.index[48:76].values.astype(int)
df7['Type of Coffee'] = 5
for i in iterator:
    for j in index_toChange:
        if i == j:
            df7.loc[:,'Type of Coffee'][i] = 'Robusta'
            break
        
        df7.loc[:,'Type of Coffee'][i] = 'Arabica'
df_pricePaidGrowers = CleaningCalendarYearGrowers (df7,'Price paid to Growers in US cents/lb',[0,1,2,3,4,8,9,37,38,46,47,76])

In [14]:
df8 = pd.read_csv('Retail_Prices.csv')
df_RetailPrices = CleaningCalendarYearRetail(df8,'Retail Prices of Roasted Coffee in US$/lb',[0,1,2,3,4,27,34,35,36])


In [15]:
print(df_totalProduction.shape,df_DomesticConsumption.shape,df_grossOpeningStock.shape,df_ExportsCropYear.shape)
print(df_ExportsCalendarYear.shape,df_Imports.shape,df_ReExports.shape)
print(df_pricePaidGrowers.shape,df_RetailPrices.shape)

(1650, 5) (1650, 5) (1650, 5) (1650, 5)
(1650, 3) (1050, 3) (1050, 3)
(1950, 4) (840, 3)


In [16]:
table1 = df_totalProduction.merge(df_DomesticConsumption,how='outer',on=['Countries','Type of Coffee','Crop Year','Crop Group']).merge(
    df_grossOpeningStock,how='outer',on=['Countries','Type of Coffee','Crop Year','Crop Group']).merge(df_ExportsCropYear,how='outer',on=['Countries','Type of Coffee','Crop Year','Crop Group'])
table2 = df_ExportsCalendarYear
table3 = df_Imports.merge(df_ReExports,how='outer',on=['Countries','Year'])
table4 = df_pricePaidGrowers
table5 = df_RetailPrices

In [17]:
table1

,Countries,Type of Coffee,Crop Group,Crop Year,Total Production (x1000 60kg) bags,Domestic Consumption (x1000 60kg) bags,Gross Opening Stock (x1000 60kg) bags,Exports Crop Year in (x1000 60kg) bags
0,Angola,Robustica/Arabica,April Group,1990/91,50,20,210,79
1,Bolivia (Plurinational State of),Arabica,April Group,1990/91,123,25,54,112
2,Brazil,Arabica/Robusta,April Group,1990/91,27286,8200,26926,17863
3,Burundi,Arabica/Robusta,April Group,1990/91,487,2,77,412
4,Ecuador,Arabica/Robusta,April Group,1990/91,1504,350,724,1628
...,...,...,...,...,...,...,...,...
1645,Trinidad & Tobago,Robusta,October Group,2019/20,12,10,0,2
1646,Uganda,Robustica/Arabica,October Group,2019/20,5509,254,900,5355
1647,Venezuela,Arabica,October Group,2019/20,650,1275,0,0
1648,Viet Nam,Robustica/Arabica,October Group,2019/20,30487,2650,5200,26537


In [18]:
table4['Price paid to Growers in US cents/lb'] = table4['Price paid to Growers in US cents/lb'].astype('float')

In [19]:
# table1.to_csv('path.../table1.csv',index=False)
# table2.to_csv('path.../table2.csv',index=False)
# table3.to_csv('path.../table3.csv',index=False)
# table4.to_csv('path.../table4.csv',index=False)
# table5.to_csv('path.../table5.csv',index=False)